In [1]:
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras import regularizers, initializers

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import os

save_dir = os.path.join(os.getcwd(), 'saved_models_homework')
model_name = "ConvNet_without_PCA"

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv(r"/home/mateusz/Desktop/18_dl/fashion-mnist_train.csv")
test_data = pd.read_csv(r"/home/mateusz/Desktop/18_dl/fashion-mnist_test.csv")

train_data = train_data.sample(frac=1)
test_data = test_data.sample(frac=1)

scaller = MinMaxScaler()

In [3]:
x_train = train_data[train_data.columns[1:]]
y_train = train_data[train_data.columns[0]]

#x_train = scaller.fit_transform(x_train)

x_test = test_data[test_data.columns[1:]]
y_test = test_data[test_data.columns[0]]


##output encoding
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1))
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1))



## reshaping and making data from 0 to 1
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

x_train = np.array(x_train).reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_train /= 255

x_test = np.array(x_train).reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_train.astype('float32')
x_test /= 255

##model params
params ={
    "input_shape" : input_shape,
    "classes" : y_train.shape[1],
    "learning_rate" : 0.0115,
    "dropout_rate" : 0.1,
    "logs" : "/home/mateusz/Desktop/18_dl/logs",
    "epochs" : 30
}


/home/mateusz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mateusz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto

In [4]:
### model creation

def create_convnet(params):
    model = Sequential()
    
    #first layer
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same",
                     input_shape=(params["input_shape"])))
    #model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(params["dropout_rate"]))
    
    ##second layer + maxpool
    model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), activation='relu', padding="same"))
    #model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(params["dropout_rate"]))
    
    model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), activation='relu', padding="same"))
    #model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(params["dropout_rate"]))
    
    
    ##Flatten layer
    model.add(Flatten())
    ##Dense
    model.add(Dense(64, activation="relu", kernel_initializer = 'random_uniform',
                bias_initializer='zeros'))
    model.add(Dropout(params["dropout_rate"]))
    model.add(Dense(128, activation="relu", kernel_initializer = 'random_uniform',
                bias_initializer='zeros'))
    model.add(Dropout(params["dropout_rate"]))
    model.add(Dense(64, activation="relu", kernel_initializer = 'random_uniform',
                bias_initializer='zeros'))
    model.add(Dropout(params["dropout_rate"]))
    model.add(Dense(params["classes"], activation="softmax"))
    
    ##compile
    optimizer = RMSprop(lr=params["learning_rate"], decay=5e-3/params["epochs"])
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['categorical_accuracy', "accuracy"])
    return model

In [5]:
## model fitting

model = create_convnet(params)
tensorboard = TensorBoard(log_dir=params["logs"], histogram_freq=0, write_graph=True, write_images=False)

W0831 23:53:40.588973 140470288582464 deprecation_wrapper.py:119] From /home/mateusz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 23:53:40.606158 140470288582464 deprecation_wrapper.py:119] From /home/mateusz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 23:53:40.611244 140470288582464 deprecation_wrapper.py:119] From /home/mateusz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0831 23:53:40.632441 140470288582464 deprecation_wrapper.py:119] From /home/mateusz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.place

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

In [7]:
history = model.fit(x_train, 
                    y_train, 
                    validation_data=(x_test, y_test), 
                    verbose=1, 
                    epochs=25, 
                    batch_size=700, 
                    callbacks=[tensorboard])

W0831 23:53:41.552275 140470288582464 deprecation.py:323] From /home/mateusz/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0831 23:53:42.350422 140470288582464 deprecation_wrapper.py:119] From /home/mateusz/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0831 23:53:42.351233 140470288582464 deprecation_wrapper.py:119] From /home/mateusz/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Train on 48000 samples, validate on 12000 samples
Epoch 1/25
48000/48000 [==============================] - 89s 2ms/step - loss: 2.1743 - categorical_accuracy: 0.2870 - acc: 0.2870 - val_loss: 0.9498 - val_categorical_accuracy: 0.5893 - val_acc: 0.5893
Epoch 2/25
48000/48000 [==============================] - 89s 2ms/step - loss: 0.9311 - categorical_accuracy: 0.6522 - acc: 0.6522 - val_loss: 0.5865 - val_categorical_accuracy: 0.7485 - val_acc: 0.7485
Epoch 3/25
48000/48000 [==============================] - 89s 2ms/step - loss: 0.5958 - categorical_accuracy: 0.7734 - acc: 0.7734 - val_loss: 0.4576 - val_categorical_accuracy: 0.8413 - val_acc: 0.8413
Epoch 4/25
48000/48000 [==============================] - 89s 2ms/step - loss: 0.4815 - categorical_accuracy: 0.8232 - acc: 0.8232 - val_loss: 0.4234 - val_categorical_accuracy: 0.8455 - val_acc: 0.8455
Epoch 5/25
48000/48000 [==============================] - 89s 2ms/step - loss: 0.4207 - categorical_accuracy: 0.8470 - acc: 0.8470 - val_l

In [8]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)     

Saved trained model at /home/mateusz/Desktop/18_dl/saved_models_homework/ConvNet_without_PCA 
